# How to build simple Chatbot with stored memory using LangChain
* Simple Chatbot LLM App
    * Will be able to have conversation
    * Will remember previous interactions: will have memory
    * Will be able to store memory in json file


## Concepts included
* Chat Model vs. LLM Model:
    *  Chat Model is based around messages
    *  LLM Model is based around raw text
* Chat History: allows Chat Model to remember previous interactions

In [1]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [2]:
MODEL_GPT = 'gpt-4o-mini'

In [3]:
# !pip list | grep langchain
!pip list | findstr langchain

langchain                                0.3.13
langchain-chroma                         0.1.4
langchain-community                      0.3.13
langchain-core                           0.3.43
langchain-experimental                   0.3.4
langchain-google-community               2.0.7
langchain-groq                           0.2.5
langchain-openai                         0.2.14
langchain-text-splitters                 0.3.4
langchainhub                             0.1.21


## Connect with LLM and start conversation with it

In [4]:
from langchain_openai import ChatOpenAI

# chatbot = ChatOpenAI(model="gpt-3.5-turbo")
chatbot = ChatOpenAI(model=MODEL_GPT)

In [5]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favorite color is blue."),
]

### Call ChatModel (LLM)

In [6]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content="That's great! Blue is a calming and soothing color. Do you have a specific shade of blue that you like the most, or do you enjoy all shades?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 13, 'total_tokens': 46, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-62a2d028-a959-4e29-b5ba-48d5089c283c-0', usage_metadata={'input_tokens': 13, 'output_tokens': 33, 'total_tokens': 46, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

### Track operation in LangSmith
* [Open LangSmith here](https://smith.langchain.com/)

## Check if Chatbot remembers our favorite color

In [7]:
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I don't have access to personal information about you, so I can't know your favorite color. However, if you tell me what it is, I'd love to hear about it!", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 13, 'total_tokens': 49, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_06737a9306', 'finish_reason': 'stop', 'logprobs': None}, id='run-26cf6d36-c2a3-482f-b120-122ae00fb1b5-0', usage_metadata={'input_tokens': 13, 'output_tokens': 36, 'total_tokens': 49, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}})

## Add memory to Chatbot

In [8]:
from langchain import LLMChain
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.prompts import HumanMessagePromptTemplate
from langchain_core.prompts import MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from langchain.memory import FileChatMessageHistory

In [9]:
memory = ConversationBufferMemory(
    chat_memory=FileChatMessageHistory("messages.json"),
    memory_key="messages",
    return_messages=True
)

C:\Users\Pavel\AppData\Local\Temp\ipykernel_7812\3060277768.py:1: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(


In [10]:
prompt = ChatPromptTemplate(
    input_variables=["content", "messages"],
    messages=[
        MessagesPlaceholder(variable_name="messages"),
        HumanMessagePromptTemplate.from_template("{content}")
    ]
)

In [11]:
chain = LLMChain(
    llm=chatbot,
    prompt=prompt,
    memory=memory
)

C:\Users\Pavel\AppData\Local\Temp\ipykernel_7812\3747834771.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [12]:
chain.invoke("hello!")

{'content': 'hello!',
 'messages': [],
 'text': 'Hello! How can I assist you today?'}

In [13]:
chain.invoke("my name is Julio")

{'content': 'my name is Julio',
 'messages': [HumanMessage(content='hello!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={})],
 'text': 'Nice to meet you, Julio! How can I help you today?'}

In [14]:
chain.invoke("what is my name?")

{'content': 'what is my name?',
 'messages': [HumanMessage(content='hello!', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='my name is Julio', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Nice to meet you, Julio! How can I help you today?', additional_kwargs={}, response_metadata={})],
 'text': 'Your name is Julio. What would you like to talk about?'}

### Check file messages.json in root directory